# Vector Database Setup and Exploration

This notebook explores ChromaDB for our LLM long-term memory project.

## Goals:
- Install and setup ChromaDB
- Create a simple collection
- Add some sample data
- Test basic similarity search
- Understand how embeddings work

## What we're building:
A memory system that stores conversations and retrieves similar past interactions.

In [6]:
# Here we are importing usefull libraries to start with..
import chromadb
import os
from datetime import datetime

#lets check if imported


print("Libraries imported successfully!")
print(f"ChromaDB version: {chromadb.__version__}")
print(f"Current time: {datetime.now()}")


Libraries imported successfully!
ChromaDB version: 1.0.20
Current time: 2025-08-29 14:14:53.221524


In [7]:
# we will create a chromadb client now
client = chromadb.Client()

# Try to get existing collection or create new one
try:
    collection = client.get_collection(name="conversation_memory")
    print("Found existing collection!")
except:
    collection = client.create_collection(
        name="conversation_memory",
        metadata={"description": "Stores conversation history for LLM memory and have a large context over conversations."}
    )
    print("Created new collection!")

print(f"Collection name: {collection.name}")
print(f"Collection count: {collection.count()}")
print("Ready to store memories!")

Created new collection!
Collection name: conversation_memory
Collection count: 0
Ready to store memories!


In [8]:
# Now we will try to add some sample conversation so that we can use for testing and creating our agent...
sample_conversations = [
    "User asked about machine learning basics and showed interest in neural networks",
    "User wants to learn Python programming and mentioned they are a beginner",
    "Discussion about building a web scraping project using BeautifulSoup",
    "User asked for help with data visualization using matplotlib and pandas",
    "Conversation about setting up a virtual environment for Python projects"
]
